In [1]:
from feos import EquationOfState, State, PhaseEquilibrium, Verbosity
from feos.parameters import PureRecord, Identifier, Parameters
from scipy.optimize import least_squares
import si_units as si
import pandas as pd
import numpy as np

In [2]:
liquid_density = pd.read_csv("hexane_liquid_density.csv")
vapor_pressure = pd.read_csv("hexane_vapor_pressure.csv")

In [3]:
type(liquid_density["temperature / K"].values * si.KELVIN)

si_units._core.SIObject

In [4]:
def liquid_density_cost(
    model: EquationOfState,
    data: pd.DataFrame,
) -> np.ndarray:
    cost = []
    for i, row in data.iterrows():
        try:
            s = State(
                model,
                temperature=row["temperature / K"] * si.KELVIN,
                pressure=row["pressure / bar"] * si.BAR,
                density_initialization="liquid",
            )
            target = row["density / kg/m3"]
            prediction = s.mass_density() / (si.KILOGRAM / si.METER**3)
            res = (prediction - target) / target
            cost.append(res)
        except Exception as _:
            cost.append(10)
    return np.array(cost)


def vapor_pressure_cost(
    model: EquationOfState,
    data: pd.DataFrame,
) -> np.ndarray:
    cost = []
    for i, row in data.iterrows():
        target = row["vapor_pressure / bar"]
        try:
            prediction = (
                PhaseEquilibrium.pure(
                    model, row["temperature / K"] * si.KELVIN
                ).vapor.pressure()
                / si.BAR
            )
            res = (prediction - target) / target
            cost.append(res)
        except Exception as _:
            cost.append(10)
    return np.array(cost)


In [6]:
# define things that are constant during optimization
identifier = Identifier(
    cas="110-54-3",
    name="hexane",
    iupac_name="hexane",
    smiles="CCCCCC",
    inchi="InChI=1/C6H14/c1-3-5-6-4-2/h3-6H2,1-2H3",
    formula="C6H14",
)
molarweight = 86.177  # g / mol
uvtheory_options = {
    "max_eta": 0.5,
    "perturbation": "WCA_TPT",
    "combination_rule": "one_fluid_psi",
    "chain_contribution": "tpt1y",
}


def eos_from_parameters(p):
    """Returns equation of state (PC-SAFT) for current parameters."""
    global identifier, molarweight
    if len(p) == 3:
        m, sigma, epsilon_k = p
        rep = 12
    elif len(p) == 4: 
        m, rep, sigma, epsilon_k = p
    pr = PureRecord(
        identifier,
        molarweight,
        m=m,
        rep=rep,
        att=6.0,
        sigma=sigma,
        epsilon_k=epsilon_k,
    )
    parameters = Parameters.new_pure(pr)
    return EquationOfState.uvtheory(parameters, **uvtheory_options)


def cost(p, liquid_density_data, vapor_pressure_data):
    """Calculates cost function for current parameters."""
    model = eos_from_parameters(p)
    cost = np.concatenate(
        [
            liquid_density_cost(model, liquid_density_data),
            vapor_pressure_cost(model, vapor_pressure_data),
        ]
    )
    return cost

In [7]:
%%time
# initial_parameters = [3.0, 12.0, 2.0, 300.0]  # m, rep, sigma, epsilon_k
# bounds = (
#     [2.0, 12.0, 2.0, 150.0],
#     [8.0, 14.0, 10.0, 500.0],
# )  # ([lower bounds], [upper bounds])

initial_parameters = [3.0, 3.0, 350.0]  # m, sigma, epsilon_k
bounds = (
    [2.0, 2.0, 150.0],
    [8.0, 8.0, 500.0],
)  # ([lower bounds], [upper bounds])

fitted_parameters = least_squares(
    cost,
    initial_parameters,
    bounds=bounds,
    loss='huber',
    f_scale=0.05,
    args=(
        liquid_density,
        vapor_pressure,
    ),
    verbose=2,
).x

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         3.4259e+01                                    1.79e+02    
       1              3         2.4750e+01      9.51e+00       2.83e+01       1.81e+00    
       2              4         2.3374e+01      1.38e+00       2.21e+00       4.29e+00    
       3              5         2.3102e+01      2.72e-01       3.90e+00       1.26e+02    
       4              6         1.5335e+01      7.77e+00       1.01e+00       1.51e+01    
       5              8         1.3994e+01      1.34e+00       1.63e-01       1.48e+01    
       6              9         1.3205e+01      7.89e-01       3.18e-01       4.15e+01    
       7             10         1.2714e+01      4.91e-01       2.72e+00       4.31e+00    
       8             11         1.2555e+01      1.59e-01       5.21e+00       1.95e+00    
       9             12         1.1455e+01      1.10e+00       5.48e+00       1.93e+01    

In [8]:
fitted_parameters

array([  2.9013988 ,   3.88041088, 253.49667306])

In [9]:
model = eos_from_parameters(fitted_parameters)
print(
    f"MARD rho_liq: {np.mean(np.abs(liquid_density_cost(model, liquid_density))) * 100:.2f} %"
)
print(
    f"MARD p_sat. : {np.mean(np.abs(vapor_pressure_cost(model, vapor_pressure))) * 100:.2f} %"
)

MARD rho_liq: 1.12 %
MARD p_sat. : 6.37 %


In [10]:
State.critical_point(model)

|temperature|density|
|-|-|
|523.44756 K|2.29457 kmol/m³|